In [1]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import os
import datetime as dt
from shapely import wkt
from shapely.geometry import Point, Polygon
import geopandas as gpd


In [9]:
from import_functions import readin_fis_biomass
# And change jupyter settings to auto-reload these functions before each instance running them
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
repo_path = Path('/Users/etriesch/dev/forest-biomass-modeling/')
data_path = repo_path / 'data/'
data_clean_path = repo_path / 'data/clean/'
geo_crs = 'epsg:4326'
proj_crs = '+proj=cea'

# read in report to year mapping

# read biomass data

In [25]:
# get filenames
path = data_path / 'biomass'
files = [f for f in os.listdir(path) 
           if f.endswith('.xml')]
files.sort()
print('files to read:', len(files))

files to read: 6


In [29]:
biodf_raw = pd.DataFrame()
for f in tqdm(files):
    filepath = str(data_path/'biomass'/ f)
    chunk = readin_fis_biomass(filepath)
    biodf_raw = pd.concat([chunk, biodf_raw])

100%|█████████████████████████████████████████████| 6/6 [00:12<00:00,  2.16s/it]

could not load file, /Users/etriesch/dev/forest-biomass-modeling/data/biomass/pa_to_wy.xml


In [28]:
biodf_raw.shape

(8433, 11)